In [8]:
import mlflow.pyfunc
import mlflow 
import tesserocr
from tesserocr import PyTessBaseAPI
import os
import re
import time

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
import pandas as pd 
df_input = pd.DataFrame({
    "path":["96486_image_19.png"],
    "lang":["eng"],
    "psm":[tesserocr.PSM.SINGLE_BLOCK]
})

In [4]:
class ocrmodel(mlflow.pyfunc.PythonModel):

    def __init__(self, path):
        self.path = path

    def predict(self, context, model_input):
        result = []
        for index in range(len(model_input)):
        
            api = PyTessBaseAPI(lang=model_input['lang'][index], psm=model_input['psm'][index])
            api.SetImageFile(model_input['path'][index])
            txt=api.GetUTF8Text()
            result.append(txt)
        # final_list.append('\n'.join(str(api.GetUTF8Text()).splitlines()))
        
        return txt
#         return model_input.apply(lambda column: column + self.n)


In [5]:
from mlflow.models.signature import infer_signature


In [6]:
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlflow.db'
experiment_name = "ocrmodel"
mlflow.set_experiment(experiment_name)

2020/09/30 10:51:23 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2020/09/30 10:51:23 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

INFO: 'ocrmodel' does not exist. Creating a new experiment


In [16]:
with mlflow.start_run(run_name="ocrmodel") as run:
   
    tic = time.time()
    ocr_model = ocrmodel(df_input)
    duration_training = time.time() - tic
#     run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    tic = time.time()
    model_path = "ocr_model2"
    mlflow.pyfunc.save_model(path=model_path, python_model=ocr_model)
    loaded_model = mlflow.pyfunc.load_pyfunc(model_path)
    model_output=loaded_model.predict(df_input)
    
#     ocrmodel.predict()
    duration_prediction = time.time() - tic
    mlflow.log_metric("Load Model Time",duration_training)
    mlflow.log_metric("OCR Time", duration_prediction)
    mlflow.log_param("Language",df_input['lang'][0])
    mlflow.log_param("psm",df_input['psm'][0])
#     mlflow.pyfunc.log_model(loaded_model, "model")
    mlflow.end_run()
    

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
<ipython-input-16-82fff497a71d>:11: DeprecationWarning: .. Warning:: ``mlflow.pyfunc.load_pyfunc`` is deprecated since 1.0. This method will be removed in a near future release. Use ``mlflow.pyfunc.load_model`` instead.
  loaded_model = mlflow.pyfunc.load_pyfunc(model_path)
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transa

In [46]:
class ocrmodel(mlflow.pyfunc.PythonModel):

    def __init__(self, lang,psm):
        self.lang = lang
        self.psm = psm
    def ocr (self,path): 
        print(path)
        api = PyTessBaseAPI(lang=self.lang, psm=self.psm)
        api.SetImageFile(path)
        txt=api.GetUTF8Text()
        return txt

    def predict(self, context, model_input):

        # final_list.append('\n'.join(str(api.GetUTF8Text()).splitlines()))
        return model_input['path'].apply(lambda column: self.ocr(column))
#         return model_input.apply(lambda column: column + self.n)


In [47]:
with mlflow.start_run(run_name="ocrmodel") as run:
    
    tic = time.time()
    ocr_model = ocrmodel(lang='eng',psm=tesserocr.PSM.SINGLE_BLOCK)
    duration_training = time.time() - tic
#     run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    tic = time.time()
    model_path = "ocr_model12"
    mlflow.pyfunc.save_model(path=model_path, python_model=ocr_model)
    loaded_model = mlflow.pyfunc.load_pyfunc(model_path)
    model_input= pd.DataFrame({"path":["96486_image_19.png","96486_image_19.png"]})
    
    model_output=loaded_model.predict(model_input)
    
#     ocrmodel.predict()
    duration_prediction = time.time() - tic
    mlflow.log_metric("Load Model Time",duration_training)
    mlflow.log_metric("OCR Time", duration_prediction)
    mlflow.log_param("Language",'eng')
    mlflow.log_param("psm",tesserocr.PSM.SINGLE_BLOCK)
#     mlflow.pyfunc.log_model(loaded_model, "model")
    mlflow.end_run()
    

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
<ipython-input-47-8a0e0c24fb40>:11: DeprecationWarning: .. Warning:: ``mlflow.pyfunc.load_pyfunc`` is deprecated since 1.0. This method will be removed in a near future release. Use ``mlflow.pyfunc.load_model`` instead.
  loaded_model = mlflow.pyfunc.load_pyfunc(model_path)


96486_image_19.png
96486_image_19.png


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [48]:
model_output

0    COLUMBIAASIA\nColumbia Asia Hospital Seremban\...
1    COLUMBIAASIA\nColumbia Asia Hospital Seremban\...
Name: path, dtype: object

In [43]:
 df = pd.DataFrame({"path":[1,2,2,3,4,5]})

In [44]:
df

,path
0,1
1,2
2,2
3,3
4,4
5,5


In [45]:
df['path'].apply(lambda column: column + 2)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    3
1    4
2    4
3    5
4    6
5    7
Name: path, dtype: int64

In [1]:
import tesserocr
import time 
from tesserocr import PyTessBaseAPI
images=["191003_112000_000-278585-4_300888_FB_FGL.PDF_9.png","Packing List 110820pdf_1.png","96486_image_19.png"]
api = PyTessBaseAPI(lang='eng', psm=tesserocr.PSM.SINGLE_BLOCK)
t1= time.time()
for path in images: 
    api.SetImageFile(path)
    txt=api.GetUTF8Text()
print(time.time()-t1)

3.0385541915893555


In [3]:
import tesserocr
import time 
from tesserocr import PyTessBaseAPI
images=["191003_112000_000-278585-4_300888_FB_FGL.PDF_9.png","Packing List 110820pdf_1.png","96486_image_19.png"]
t1= time.time()
for path in images: 
    api = PyTessBaseAPI(lang='eng', psm=tesserocr.PSM.SINGLE_BLOCK)
    api.SetImageFile(path)
    txt=api.GetUTF8Text()
print(time.time()-t1)

3.2324917316436768


In [15]:
import requests

url = "http://localhost:5001/invocations"
payload= "{\"columns\": [\"path\"], \"data\": [[\"test/20200221102506227_FMG_booking_confirmationpdf_1.png\"], [\"test/20200221102506227_FMG_booking_confirmationpdf_2.png\"]]}"
# '{"columns": ["path"],"data": [[\'test/20200221102506227_FMG_booking_confirmationpdf_1.png\'], [\'test/20200221102506227_FMG_booking_confirmationpdf_2.png\']]}'
# payload = "{\n    \"columns\": [\n        \"path\"\n    ],\n    \"data\": [\n[\n\"/home/ai-machine/ICE-Ai/Data/maryam/CNN_ICEAI/cnn/test_data/177323_image_0.png\"]]\n}"
headers = {
  'Content-type': 'application/json; format=pandas-split'
}

response = requests.request("POST", url, headers=headers, data = payload)

print(response.text.encode('utf8'))


b'{"error_code": "BAD_REQUEST", "message": "Encountered an unexpected error while evaluating the model. Verify that the serialized input Dataframe is compatible with the model for inference.", "stack_trace": "Traceback (most recent call last):\\n  File \\"/home/ai-machine/.local/lib/python3.8/site-packages/mlflow/pyfunc/scoring_server/__init__.py\\", line 213, in transformation\\n    raw_predictions = model.predict(data)\\n  File \\"/home/ai-machine/.local/lib/python3.8/site-packages/mlflow/pyfunc/__init__.py\\", line 425, in predict\\n    return self._model_impl.predict(data)\\n  File \\"/home/ai-machine/.local/lib/python3.8/site-packages/mlflow/pyfunc/model.py\\", line 254, in predict\\n    return self.python_model.predict(self.context, model_input)\\n  File \\"/home/ai-machine/tesseractserving/mlflowtesseract.py\\", line 35, in predict\\n    def start(image_path,lang):\\n  File \\"/home/ai-machine/.local/lib/python3.8/site-packages/pandas/core/series.py\\", line 3848, in apply\\n   